In [ ]:
# Generate random id- numbers
import random

vk_users_n=710000000
random.seed(7)
id_list_raw=[random.randrange(1, vk_users_n+1, 1) for i in range(200000)]  # generate 200000 rnd ids

In [ ]:
# ==== Limits on wall.get calls - 5000 calls per day ======
id_chunk=id_list_raw[0:5000]

In [ ]:
# Authorization in VK

import vk
import json
import requests

access_token=#your token
v='5.154'

In [ ]:
import hashlib

# Hashing ids

def hash_user_id(user_id):
    return hashlib.sha256(f"user_{user_id}".encode()).hexdigest()

def hash_post_id(post_id):
    return hashlib.sha256(f"post_{post_id}".encode()).hexdigest()

def parse_results(id_, post, post_df):
    post_list=[]
    post_list.append(hash_user_id(id_))  # hashed user ID
    post_list.append(hash_post_id(post.get('id', -9999)))  # hashed post ID
    full_date=datetime.fromtimestamp(post['date'])
    post_list.append(datetime.date(full_date))
    text_=post['text'].replace("\n"," ")
    text=text_.replace("|"," ")
    post_list.append(text)


    get_record=False 
    if 'attachments' in post:
        for att in post['attachments']:    
            if (att['type']=='link'):
                post_list.append(att.get('link', {'title':'',}).get('title', ''))
                post_list.append(att.get('link', {'description':'',}).get('description', ''))
                get_record=True
    if not get_record:
        post_list.append("")
        post_list.append("")


    # If it is a repost with a comment, there will be several elements in the history, with several texts. We only take the user's text (the first one).
    if 'copy_history' in post:
        for att in post['copy_history']:
            if att.get('text', '')!=u"Запись удалена":
                post_list.append(att.get('text', '').replace("\n"," "))
                break
    else:
        post_list.append('')


    #==== Check that all list items are not empty (except hash_user_id, date). They may be empty, for example, if only a photo has been added
    k=0
    for i in range(3, len(post_list)-3):
        if len(str(post_list[i]))>0:
            k=k+1

    if k>0:
    #======Only if there is text/ repost of a post or story, we collect information about views/likes/reposts/ =================
        post_list.append(post.get('views', {'count':'',}).get('count', ''))
        post_list.append(post.get('likes', {'count':'',}).get('count', ''))
        post_list.append(post.get('comments', {'count':'',}).get('count', ''))
        post_list.append(post.get('reposts', {'count':'',}).get('count', ''))


        post_df = pd.concat([post_df, pd.DataFrame(pd.Series(post_list, index=post_df.columns)).T], axis=0, ignore_index=True)
        pbar_concat.set_description("Last added for  %s" % id_)
        pbar_concat.update()
    return post_df

In [ ]:
# Main 

start_i=0

day_limit=5000
res_folder=#/type you dir for results/

import pandas as pd
import time
from datetime import datetime
format = '%Y-%m-%d'

post_df=pd.DataFrame(columns=['user_idh', 'post_idh', 'date', 'text', 'link_title', 'link_description', 'repost_text', 'views', 'likes', 'comments', 'reposts'])
err=''

# progress bar tqdm
from tqdm.notebook import tqdm
pbar = tqdm(id_chunk)
pbar.set_description("Processing id_s")

pbar_concat = tqdm()
pbar_concat.set_description("Last added for None")

requests_tick = datetime.now()
td_hold = (1.05/3)

for i, id_ in enumerate(pbar):
    
    if err=='limit':
        break
    
    offset=0
    
    while True:
        
        td = (datetime.now() - requests_tick).total_seconds()
        if td < td_hold:
            time.sleep(td_hold - td)
        
        requests_tick = datetime.now()        
        try:
            q = requests.get('https://api.vk.com/method/wall.get', params={
                        'access_token':access_token,  
                        'v':v,
                        'owner_id':id_,
                        'count':100,
                        'offset':offset,
                        'filter':'owner'
                        }).json()['response']
        except Exception as error:
            try:
                if q['error']['error_code']==29:
                    err='limit'
                    print ('Rate limit reached')
                    print (i)
                    break
            except:
                break
        
        if len(q['items']) == 0:
            break       
        offset=offset+100
        
        
        for post in q['items']:
             
            full_date_=datetime.fromtimestamp(post['date'])
            full_date = full_date_.strftime(format)
            post_df=parse_results(id_, post, post_df)

name_end=start_i+id_chunk.index(id_)+1
name='part_'+str(start_i)+'-'+str(name_end)+'.csv'
post_df.to_csv(res_folder+name, sep='|', encoding='utf-8')   